In [1]:
#import xgboost
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
#from matplotlib import style
#style.use('fivethirtyeight')
import seaborn as sns
#sns.set(style = 'white', color_code=True)
import warnings
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')

In [2]:
#Load train and test data from csv files
#train_loan_df = pd.read_csv('Loan_train.csv',dtype={'Loan_ID'[2:]:int})
train_loan_df = pd.read_csv('train.csv')
test_loan_df  = pd.read_csv('test.csv')
#train_loan_df.corr()
train_loan_df.shape,test_loan_df.shape
#train_loan_df['Loan_ID'] = [pd.to_numeric(x[2:],errors='ignore') for x in train_loan_df['Loan_ID'] ]
# train_loan_df.info()
# train_loan_df.head()

((614, 13), (367, 12))

In [3]:
#Gender Column
train_loan_df['Gender'] = train_loan_df['Gender'].fillna('Male')
test_loan_df['Gender'] = test_loan_df['Gender'].fillna('Male')

gender_map = {'Male':0,'Female':1}

train_loan_df['Gender'] = train_loan_df['Gender'].map(gender_map)
test_loan_df['Gender'] = test_loan_df['Gender'].map(gender_map)

#Married Column
train_loan_df['Married'] = train_loan_df['Married'].fillna('Yes')
test_loan_df['Married'] = test_loan_df['Married'].fillna('Yes')

married_map = {'Yes':1,'No':0}

train_loan_df['Married'] = train_loan_df['Married'].map(married_map)
test_loan_df['Married'] = test_loan_df['Married'].map(married_map)

#Dependent Column
train_loan_df['Dependents'] = train_loan_df['Dependents'].fillna('2')
test_loan_df['Dependents'] = test_loan_df['Dependents'].fillna('2')

#dep_map = {'0':69,'1':65,'2':75,'3+':64}
dep_map = {'0':0,'1':1,'2':2,'3+':3}

train_loan_df['Dependents'] = train_loan_df['Dependents'].map(dep_map)
test_loan_df['Dependents'] = test_loan_df['Dependents'].map(dep_map)

#Loan Amount Term
train_loan_df['Loan_Amount_Term'] = train_loan_df['Loan_Amount_Term'].fillna(train_loan_df['Loan_Amount_Term'].mean())
test_loan_df['Loan_Amount_Term'] = test_loan_df['Loan_Amount_Term'].fillna(test_loan_df['Loan_Amount_Term'].mean())

#LoanAmount
train_loan_df['LoanAmount'] = train_loan_df['LoanAmount'].fillna(train_loan_df['LoanAmount'].mean())
test_loan_df['LoanAmount'] = test_loan_df['LoanAmount'].fillna(test_loan_df['LoanAmount'].mean())

#Self Employed
train_loan_df['Self_Employed'] = train_loan_df['Self_Employed'].fillna('No')
test_loan_df['Self_Employed'] = test_loan_df['Self_Employed'].fillna('No')

self_emp_map = {'Yes':0,'No':1}

train_loan_df['Self_Employed'] = train_loan_df['Self_Employed'].map(self_emp_map)
test_loan_df['Self_Employed'] = test_loan_df['Self_Employed'].map(self_emp_map)

#Property Map
#property_map = {'Semiurban':77,'Urban':66,'Rural':62}
property_map = {'Semiurban':3,'Urban':2,'Rural':1}
train_loan_df['Property_Area'] = train_loan_df['Property_Area'].map(property_map)
test_loan_df['Property_Area']  = test_loan_df['Property_Area'].map(property_map)

#Education Column
edu_map = {'Graduate':1,'Not Graduate':0}
train_loan_df.Education = train_loan_df.Education.map(edu_map)
test_loan_df.Education = test_loan_df.Education.map(edu_map)

#Credit History Column
train_loan_df['Credit_History'] = train_loan_df['Credit_History'].fillna(1.0)
test_loan_df['Credit_History']  = test_loan_df['Credit_History'].fillna(1.0)

#New EMI Column
train_loan_df['EMI'] = train_loan_df['LoanAmount']/train_loan_df['Loan_Amount_Term']
test_loan_df['EMI'] = test_loan_df['LoanAmount']/test_loan_df['Loan_Amount_Term']


#Credit_Pos_Index
train_loan_df['Credit_Pos_Index'] = train_loan_df['Self_Employed'] + train_loan_df['Education'] + train_loan_df['Married'] + train_loan_df['Dependents']
test_loan_df['Credit_Pos_Index'] = test_loan_df['Self_Employed'] + test_loan_df['Education'] + test_loan_df['Married'] + test_loan_df['Dependents']

#Loan_Value
train_loan_df['Loan_Value'] = train_loan_df['LoanAmount']/(train_loan_df['ApplicantIncome'] + train_loan_df['CoapplicantIncome'])
test_loan_df['Loan_Value'] = test_loan_df['LoanAmount']/(test_loan_df['ApplicantIncome'] + test_loan_df['CoapplicantIncome'])

In [4]:
train_loan_df.head()
#Total Income
# train_loan_df['Total_Income'] = train_loan_df['ApplicantIncome'] + train_loan_df['CoapplicantIncome']
# test_loan_df['Total_Income'] = test_loan_df['ApplicantIncome'] + test_loan_df['CoapplicantIncome']
# train_loan_df.drop('ApplicantIncome',axis=1,inplace=True)
# train_loan_df.drop('CoapplicantIncome',axis=1,inplace=True)
# test_loan_df.drop('CoapplicantIncome',axis=1,inplace=True)
# test_loan_df.drop('ApplicantIncome',axis=1,inplace=True)
# loan_status_map = {'Y':1,'N':0}
# train_loan_df['Loan_Status'] = train_loan_df['Loan_Status'].map(loan_status_map)
# train_loan_df.drop(['Gender','Married'],axis=1,inplace=True)
# test_loan_df.drop(['Gender','Married'],axis=1,inplace=True)

#New Loan Value Column
# train_loan_df['Loan_Value'] = (train_loan_df['ApplicantIncome'] + train_loan_df['CoapplicantIncome'])/train_loan_df['LoanAmount']
# test_loan_df['Loan_Value'] = (test_loan_df['ApplicantIncome'] + test_loan_df['CoapplicantIncome'])/test_loan_df['LoanAmount']
# train_loan_df['Loan_Value'] = train_loan_df['ApplicantIncome']/train_loan_df['LoanAmount']
# test_loan_df['Loan_Value'] = test_loan_df['ApplicantIncome']/test_loan_df['LoanAmount']

#Credit Index
# train_loan_df['Credit_Index'] = train_loan_df[train_loan_df['Education']==2]['Education'] + train_loan_df[train_loan_df['Self_Employed']==0]['Self_Employed']
# train_loan_df['Credit_Index']  = train_loan_df['Credit_Index'] .fillna(0)
# test_loan_df['Credit_Index'] = test_loan_df[test_loan_df['Education']==2]['Education'] + test_loan_df[test_loan_df['Self_Employed']==0]['Self_Employed']
# test_loan_df['Credit_Index']  = test_loan_df['Credit_Index'] .fillna(0)
# train_loan_df['Credit_Index'] = train_loan_df['Education'] + train_loan_df['Self_Employed']
# test_loan_df['Credit_Index'] = test_loan_df['Education'] + test_loan_df['Self_Employed']

#Negative Index
# train_loan_df['Neg_Index'] = train_loan_df['Dependents']+train_loan_df['Property_Area']
# test_loan_df['Neg_Index'] = test_loan_df['Dependents']+test_loan_df['Property_Area']

#EMI_Dep
# train_loan_df['EMI_Dep'] = train_loan_df['EMI']/train_loan_df['Dependents']
# test_loan_df['EMI_Dep'] = test_loan_df['EMI']/test_loan_df['Dependents']

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,EMI,Credit_Pos_Index,Loan_Value
0,LP001002,0,0,0,1,1,5849,0.0,146.412162,360.0,1.0,2,Y,0.406700,2,0.025032
1,LP001003,0,1,1,1,1,4583,1508.0,128.000000,360.0,1.0,1,N,0.355556,4,0.021015
2,LP001005,0,1,0,1,0,3000,0.0,66.000000,360.0,1.0,2,Y,0.183333,2,0.022000
3,LP001006,0,1,0,0,1,2583,2358.0,120.000000,360.0,1.0,2,Y,0.333333,2,0.024287
4,LP001008,0,0,0,1,1,6000,0.0,141.000000,360.0,1.0,2,Y,0.391667,2,0.023500


In [5]:
#loan_status_map = {'Y':1,'N':0}
#train_loan_df['Loan_Status'] = train_loan_df['Loan_Status'].map(loan_status_map)

#self_emp_map = {'Yes':1,'No':0}

#train_loan_df['Self_Employed'] = train_loan_df['Self_Employed'].map(self_emp_map)
#train_loan_df.head()
# train_loan_df.info()
# test_loan_df.info()
# train_loan_df.head()
# train_loan_df['Credit_history']

# train_loan_df[train_loan_df['Credit_History']==0]
# train_loan_df['Credit_History'].value_counts()
# #sns.barplot(x='AppplicantIncome', y='Self_Employed', data=train_loan_df)
# #sns.factorplot('AppplicantIncome',hue='Self_Employed',data=train_loan_df,kind='count',order=[1,0])
# #train_loan_df[train_loan_df['Dependents'].isnull()].info()

#fig,(axis1,axis2) = plt.subplots(1,2,figsize=(15,5))
#sns.countplot(x='Credit_History', data=train_loan_df, ax=axis1)
#sns.countplot(x='Property_Area', hue="Loan_Status", data=train_loan_df, order=[1,0], ax=axis1)
#sns.countplot(x='Dependents', hue="Loan_Status", data=train_loan_df, order=[1,0], ax=axis2)
# #sns.countplot(x='Credit_History', data=train_loan_df, ax=axis1)
#sns.factorplot('Dependents','Loan_Status', data=train_loan_df,size=4,aspect=3)
#sns.factorplot('Dependenta','Loan_Status', data=train_loan_df,size=4,aspect=3)
# #sns.countplot(x='Credit_History', data=train_loan_df, ax=axis1)
# sns.countplot(x='Education', hue="Credit_History", data=train_loan_df, order=[1,0], ax=axis3)
# #sns.countplot(x='Credit_History', data=train_loan_df, ax=axis1)
#sns.factorplot('Dependents','Credit_History', data=train_loan_df,size=4,aspect=3)
# # average_term = train_loan_df[["Loan_Amount_Term", "Loan_Status"]].groupby(['Loan_Amount_Term'],as_index=False).mean()
# # sns.barplot(x='Loan_Amount_Term', y='Loan_Status', data=average_term,ax=axis1)

# train_loan_df['Loan_Amount_Term'] = train_loan_df['Loan_Amount_Term'].fillna(train_loan_df['Loan_Amount_Term'].mean())
# average_term1 = train_loan_df[["Loan_Amount_Term", "Loan_Status"]].groupby(['Loan_Amount_Term'],as_index=False).mean()
#sns.barplot(x='AppplicantIncome', y='Self_Employed', data=train_loan_df)

In [6]:
#Credit History Model
#X_credit_train = train_loan_df[(train_loan_df['Credit_History'].notnull())][['ApplicantIncome','Dependents','Property_Area','Education','Self_Employed']]
# X_credit_train = train_loan_df[(train_loan_df['Credit_History'].notnull())][['ApplicantIncome','LoanAmount']]
# y_credit_train = train_loan_df[(train_loan_df['Credit_History'].notnull())]['Loan_Status']
#credit_test = train_loan_df[train_loan_df['Credit_History'].isnull()][['ApplicantIncome','Dependents','Property_Area','Education','Self_Employed']]
# credit_test = train_loan_df[train_loan_df['Credit_History'].isnull()][['ApplicantIncome','LoanAmount']]
# credit_test.head()

In [7]:
# from sklearn.linear_model import LogisticRegression
# logReg = LogisticRegression()
# logReg.fit(X_credit_train,y_credit_train)
# credit_pred = logReg.predict(credit_test)
# credit_pred
#train_loan_df["Credit_History"][np.isnan(train_loan_df["Credit_History"])] = credit_pred

In [8]:
X_train = train_loan_df.drop(['Loan_Status','Loan_ID'],axis=1)
Y_train = train_loan_df['Loan_Status']
X_test  = test_loan_df.drop('Loan_ID',axis=1)
X_train.info(),X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 14 columns):
Gender               614 non-null int64
Married              614 non-null int64
Dependents           614 non-null int64
Education            614 non-null int64
Self_Employed        614 non-null int64
ApplicantIncome      614 non-null int64
CoapplicantIncome    614 non-null float64
LoanAmount           614 non-null float64
Loan_Amount_Term     614 non-null float64
Credit_History       614 non-null float64
Property_Area        614 non-null int64
EMI                  614 non-null float64
Credit_Pos_Index     614 non-null int64
Loan_Value           614 non-null float64
dtypes: float64(6), int64(8)
memory usage: 67.2 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 14 columns):
Gender               367 non-null int64
Married              367 non-null int64
Dependents           367 non-null int64
Education            367 non-null int64
Self_Em

(None, None)

In [9]:
from sklearn.linear_model import LogisticRegression
logReg = LogisticRegression()
logReg.fit(X_train,Y_train)
Y_pred = logReg.predict(X_test)
logReg.score(X_train,Y_train)

0.81107491856677527

In [10]:
# from sklearn import svm
# svc = svm.SVC()
# svc.fit(X_train,Y_train)
# Y_pred = svc.predict(X_test)
# svc.score(X_train,Y_train)

In [11]:
# get Correlation Coefficient for each feature using Logistic Regression
coeff_df = pd.DataFrame(train_loan_df.columns.delete(0))
coeff_df.columns = ['Features']
coeff_df["Coefficient Estimate"] = pd.Series(logReg.coef_[0])

# preview
coeff_df

,Features,Coefficient Estimate
0,Gender,-0.003008
1,Married,0.286436
2,Dependents,-0.130881
3,Education,0.182623
4,Self_Employed,-0.187441
5,ApplicantIncome,0.000006
6,CoapplicantIncome,-0.000051
7,LoanAmount,-0.000502
8,Loan_Amount_Term,-0.004198
9,Credit_History,2.958619


In [12]:
from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
import numpy as np

np.random.seed(123)

clf1 = LogisticRegression()
clf2 = RandomForestClassifier()
clf3 = GaussianNB()

print('5-fold cross validation:\n')

for clf, label in zip([clf1, clf2, clf3], ['Logistic Regression', 'Random Forest', 'naive Bayes']):

    scores = cross_validation.cross_val_score(clf, X_train, Y_train, cv=10, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

5-fold cross validation:

Accuracy: 0.80 (+/- 0.04) [Logistic Regression]
Accuracy: 0.76 (+/- 0.06) [Random Forest]
Accuracy: 0.79 (+/- 0.04) [naive Bayes]


In [13]:
# from sklearn import metrics
# C_start = 0.1
# C_end = 5
# C_inc = 0.1

# C_values, recall_scores = [], []

# C_val = C_start
# best_recall_score = 0
# while (C_val < C_end):
#     C_values.append(C_val)
#     lr_model_loop = LogisticRegression(C=C_val, random_state=42)
#     lr_model_loop.fit(X_train, Y_train.ravel())
#     lr_predict_loop_test = lr_model_loop.predict(X_test)
#     recall_score = metrics.recall_score(Y_pred, lr_predict_loop_test)
#     recall_scores.append(recall_score)
#     if (recall_score > best_recall_score):
#         best_recall_score = recall_score
#         best_lr_predict_test = lr_predict_loop_test
        
#     C_val = C_val + C_inc

# best_score_C_val = C_values[recall_scores.index(best_recall_score)]
# print("1st max value of {0:.3f} occured at C={1:.3f}".format(best_recall_score, best_score_C_val))

# %matplotlib inline 
# plt.plot(C_values, recall_scores, "-")
# plt.xlabel("C value")
# plt.ylabel("recall score")

In [14]:
# from sklearn.linear_model import LogisticRegression
# lr_model =LogisticRegression( class_weight="balanced", C=best_score_C_val, random_state=42)
# lr_model.fit(X_train, y_train.ravel())
# y_pred = lr_model.predict(X_test)
# lr_model.score(X_train,y_train)

In [15]:
# from sklearn.ensemble import RandomForestClassifier
# rf = RandomForestClassifier(n_estimators=40)
# rf.fit(X_train,Y_train)
# Y_pred = rf.predict(X_test)
# rf.score(X_train,Y_train)

In [16]:
submission = pd.DataFrame({
        'Loan_ID':test_loan_df['Loan_ID'],
        'Loan_Status':Y_pred
    })
submission.to_csv('Loan_Prediction.csv',index=False)